# Document Clustering
### With Preprocessing

In [1]:
import json
import os
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import spacy
# import coreferee
# nlp = spacy.load('en_core_web_trf')
# nlp.add_pipe('coreferee')
nlp = spacy.load("en_core_web_sm")

2022-08-28 21:30:42.087877: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-28 21:30:42.087966: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Define Parameters

In [16]:
n_clusters = 5 
n = 5 #Transcripts to use
max_df=0.9
min_df=0.0001
n_samples = 10

## Load Data

In [9]:
# pos = ["CCONJ","NUM","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
# pos = ["ADJ","ADP","ADV","AUX","CCONJ","DET","INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","VERB","X"]
pos = ["ADJ","AUX","CCONJ","DET","INTJ","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
def clean_text(text, coref = False, lemma = True, pos_exclusions = pos):
    doc = nlp(text)

    if coref:
        new_text = ""
        for i  in range(len(doc)):
            tokens = doc[i]
            if doc._.coref_chains.resolve(tokens):
                new_text += doc._.coref_chains.resolve(tokens)[0].text
            else:
                new_text += tokens.text
            new_text+= " "
        doc = nlp(new_text)
    text = ""
    for token in doc:
        if token.pos_ not in pos_exclusions:
            text += token.lemma_ if lemma else token.text
            text += " "
    return text

In [4]:
def create_df_multi(n=25):
    list_of_text = []
    dir = 'data/aligned data/c=4'

    files = [filename for filename in os.listdir(dir)]
    sampled_files = random.choices(files,k=n)

    for filename in sampled_files:
        # choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+filename)
        data = json.load(f)
        choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+choice)
        data = json.load(f)

        for x in data:
            for y in x['TURNS']:
                text = ' '.join(y['UTTERANCES'])

                text = clean_text(text)
                list_of_text.append(text)
    df = pd.DataFrame(list_of_text)
    return df

In [5]:
df = create_df_multi(n)
print(df.shape)
df.head()

(13601, 1)


,0
0,to tonight episode of bunch of ass voice actor...
1,slur play dungeon
2,go have join as soon come in come in from work...
3,mime work
4,come to worst obviously fill in


## Vectorize Data



- Count Vectorizer
- Tfidf Vectorizer

Use both with each model

In [18]:
# tfidf=TfidfVectorizer(stop_words='english',max_df=.7,min_df=2,token_pattern=r'(?u)\b[A-Za-z]+\b')

tfidf = TfidfVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
tfidf_sparse = tfidf.fit_transform(df[0])
print(tfidf_sparse.shape)
tfidf_df = pd.DataFrame(tfidf_sparse.toarray())
tfidf_df.tail()

(13601, 2728)


,0,1,2,3,4,5,6,7,8,9,...,2718,2719,2720,2721,2722,2723,2724,2725,2726,2727
13596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
cv = CountVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
cv_sparse = cv.fit_transform(df[0])
print(cv_sparse.shape)
cv_df = pd.DataFrame(cv_sparse.toarray())
cv_df.tail()

(13601, 2728)


,0,1,2,3,4,5,6,7,8,9,...,2718,2719,2720,2721,2722,2723,2724,2725,2726,2727
13596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13600,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Define Output Functions

In [12]:
def sample_clusters(labels, n_clusters = n_clusters, n_samples = 10):
    df['labels'] = labels
    for i in range(n_clusters):
        print(f'Cluster {i}: {len(df[df.labels == i])} / {len(df)}')
        sample = df[df.labels == i].sample(n_samples, replace=True)[0]
        for x in sample:
            print('\t- ' + x[:100])
        print('\n')
        print('==='*30)
        print('\n')


## 5 Clusters

In [20]:
n_clusters = 5 
n_samples = 10

In [21]:
kmeans = KMeans(n_clusters = n_clusters, random_state= 42)
kmeans = kmeans.fit(cv_df)
sample_clusters(kmeans.labels_, n_clusters=n_clusters, n_samples=n_samples)

Cluster 0: 11438 / 13601
	- go take shot though again 
	- have inspiration dice give out give out as bonus action give out go into battle 
	- think send army up again again towards door of 
	- say time 
	- do to friend 
	- 
	- definitely one with laughter 
	- 
	- 
	- love name announce woman thing 




Cluster 1: 681 / 13601
	- point of damage attack against guy have advantage now beacon blast streak forward from hand smash in
	- go one look for be kid just go to dentist look talk real funny go go back try get back in history to
	- lava cave just look 
	- indeed so have stuff down road talk about in meantime let go ahead get catch up speed with transpire
	- so guy rip arm out of face essentially dagger come swinge out from behind doorway stab in shoulder s
	- as shit pinche everywhere plate about size too for get get inside of hang off of get piece of armor 
	- leave possibly by end of day give back gold case need look off towards ship wait go kind of need spa
	- open-- look like taver

In [22]:
kmeans = KMeans(n_clusters = n_clusters, random_state= 42)
kmeans = kmeans.fit(tfidf_df)
sample_clusters(kmeans.labels_, n_clusters=n_clusters, n_samples=n_samples)

Cluster 0: 11818 / 13601
	- 
	- all right so also suffer poison course through body point of pierce damage go double technically att
	- here dagger 
	- laugh run 
	- lot 
	- part air tunnel for 
	- for thing think create diversion let do use deck of illusion generate kind of number of sort of crea
	- kind of creature make 
	- fall surface slime already fall gather sheep all harvest for 
	- fuck shit 




Cluster 1: 539 / 13601
	- like note like 
	- as shit pinche everywhere plate about size too for get get inside of hang off of get piece of armor 
	- look at thing stand here currently 
	- see look like pile of rag in corner 
	- do perhaps stand in back then guy want go loosely way about pace to right of stone see with outcropp
	- look have appendage 
	- percy take look at 
	- just party take turn like so 
	- see through eye like 
	- look at item find 




Cluster 2: 608 / 13601
	- probably go hit on minus on 
	- hit 
	- in dream see from top down hit water right in middle of keep go 
	

## 10 Clusters

In [23]:
n_clusters = 10 
n_samples = 10

In [24]:
kmeans = KMeans(n_clusters = n_clusters, random_state= 42)
kmeans = kmeans.fit(cv_df)
sample_clusters(kmeans.labels_, n_clusters=n_clusters, n_samples=n_samples)

Cluster 0: 10904 / 13601
	-   donation for for role group from 
	- so place fly in think have check for trap 
	- 
	- want do find treasure room 
	- poke head into want get look 
	- save throw 
	- target move through hand space strength score than to hand strength score 
	- let go in storage room go up 
	- there 
	- 




Cluster 1: 408 / 13601
	- guy watch specter vanishe into shadow of tower below 
	- so see guy 
	- guy hangup mine laughter 
	- guy step in here move up towards corner be glint stay inside outside 
	- guy watch specter vanishe into shadow of tower below 
	- so so guy thank so much for watch also as reminder be do kickstarter for set supporter for of kickst
	- guy want send in bulette right now 
	- guy manage make way back manage recover gold from dockmaster get back to ship hear rest of party ret
	- guy rest all right so first top at top of initiative order 
	- guy rest all right so first top at top of initiative order 




Cluster 2: 4 / 13601
	- sound of storm movement

In [25]:
kmeans = KMeans(n_clusters = n_clusters, random_state= 42)
kmeans = kmeans.fit(tfidf_df)
sample_clusters(kmeans.labels_, n_clusters=n_clusters, n_samples=n_samples)

Cluster 0: 11721 / 13601
	- sing dex of panther-- 
	- perch on shoulder crow sound 
	- room of burst into view now see wall ceiling-- stone room be chandelier currently hang from top of c
	- want 
	- say 
	- stick with 
	- go with 
	- shirt right there so have temporary-- very have up guy ever want go to have run of shirt currently f
	- turn into animal 
	- still very 




Cluster 1: 175 / 13601
	- let choose 
	- let go get 
	- let talk about 
	- let check 
	- let find then decide later 
	- get let see 
	- let find then decide later 
	- let go 
	- man let see get lightning back roll beforehand 
	- let go in storage room go up 




Cluster 2: 256 / 13601
	- know roll go roll again 
	- thing just touch know know expect like glass shattering none of shit boop 
	- congrat on get know guy 
	- maybe know ooze wanna do 
	- mean already know here 
	- know do think do before 
	- know want ascertain 
	- seem know fuck all about 
	- kind of animal god use know kind of animal duergar worship know 

## 20 Clusters

In [26]:
n_clusters = 20 
n_samples = 10

In [27]:
kmeans = KMeans(n_clusters = n_clusters, random_state= 42)
kmeans = kmeans.fit(cv_df)
sample_clusters(kmeans.labels_, n_clusters=n_clusters, n_samples=n_samples)

Cluster 0: 1 / 13601
	- go use bonus action disengage from sphinx go run around whirlwind move take rope already make loop o
	- go use bonus action disengage from sphinx go run around whirlwind move take rope already make loop o
	- go use bonus action disengage from sphinx go run around whirlwind move take rope already make loop o
	- go use bonus action disengage from sphinx go run around whirlwind move take rope already make loop o
	- go use bonus action disengage from sphinx go run around whirlwind move take rope already make loop o
	- go use bonus action disengage from sphinx go run around whirlwind move take rope already make loop o
	- go use bonus action disengage from sphinx go run around whirlwind move take rope already make loop o
	- go use bonus action disengage from sphinx go run around whirlwind move take rope already make loop o
	- go use bonus action disengage from sphinx go run around whirlwind move take rope already make loop o
	- go use bonus action disengage from sphin

In [28]:
kmeans = KMeans(n_clusters = n_clusters, random_state= 42)
kmeans = kmeans.fit(tfidf_df)
sample_clusters(kmeans.labels_, n_clusters=n_clusters, n_samples=n_samples)

Cluster 0: 10341 / 13601
	- 
	- all right survive go kill dragon thing have build from scale of dragon people on plane adorn with sc
	- do for night 
	- come for 
	- Mm hmm 
	- get on 
	- boat call 
	- 
	- eye leave blade go 
	- so in dark 




Cluster 1: 242 / 13601
	- want do 
	- want do roll time 
	- really want go first 
	- want throw ass in pit cheer 
	- go down far want 
	- nearby none see at want go look for 
	- want maybe look at book leave 
	- go down go yell want live throw down arm do 
	- just want control happen 
	- start walk very slowly bee want honey 




Cluster 2: 400 / 13601
	- just forget 
	- go move just behind of within range at point 
	- so just keep sword at 
	- go rocket toward just give second think 
	- just get leave see do 
	- hear footprint of still essentially just-- 
	- then just-- fuck see then just yell at 
	- still have source up just campfire 
	- just have out just have energy 
	- just roll on phone couple time then off to side 




Cluster 3: 74 / 136